In [ ]:
# Authenticate user
import spotipy
import spotipy.util as util

def get_token(scope):
    token = util.prompt_for_user_token(
            username='',
            scope=scope,
            client_id='e74c52988f6d4bcebb36970a423d348d',
            client_secret='0edc87deae1a4611a97b6cebef262136',
            redirect_uri='http://localhost:8888/callback')
    return token
token = get_token(scope='user-top-read')
if token:
    sp = spotipy.Spotify(auth=token)

In [ ]:
# Experiment
# Get user's profile information
profile = sp.current_user()

In [ ]:
print [u'\u201impo

In [ ]:
# Experiment
# Get user's top artists
# Part 1: Function
def get_users_top_artists(limit=20, offset=0, time_range='long_term'):
    results = sp.current_user_top_artists(limit=limit, offset=offset, time_range=time_range)
    return results['items']

def get_all_users_top_artists(num_artists=50):
    num_artists = min(num_artists, 50)
    artists = []
    chunk_size = 20
    for index in range(0, num_artists, chunk_size):
        artists.extend(get_users_top_artists(limit=chunk_size, offset=index))
    return artists

In [ ]:
# Part 2: Print
for artist in get_users_top_artists(time_range='long_term'):
    print artist['name']

In [ ]:
# Experiment
# Part 1: Get related artists
def get_related_artists(artist):
    return sp.artist_related_artists(artist[u'id'])

In [ ]:
related_artists = {}
for artist in get_users_top_artists():
    related_artists[artist['name']] = get_related_artists(artist)

In [ ]:
Parallel(n_jobs=4)

In [ ]:
# Part 2: Print artist and similar artists
for key, val in related_artists.iteritems():
    artist_name = key
    similar_artists = val['artists']
    similar_artist_names = [artist['name'] for artist in similar_artists]
    print artist_name
    print similar_artist_names

In [ ]:
# Experiment
# Part 1: Get artist remixes
def get_remixes(artist, limit=50, offset=0):
    # Returns the query searching for artist + remix
    name = artist['name']
    
    def search_spotify_tracks(query):
        results = sp.search(q=query, type='track', limit=limit, offset=offset)
        tracks = [track for track in results['tracks']['items']]
        return tracks
     
    remix_tracks = search_spotify_tracks('{} remix'.format(name))
    return remix_tracks

def get_all_remixes(artist):
    all_remixes = []
    offset = 0
    limit = 50
    while True:
        remixes = get_remixes(artist, limit=limit, offset=offset)
        if not remixes:
            break
        all_remixes.extend(remixes)
        offset += limit
    return all_remixes

def get_tracks_remixed_by(artist):
    # filter only tracks the artist has remixed
    def artist_uris(track):
        return [a['uri'] for a in track['artists']]
    
    #remixes = [remix for remix in get_remixes(artist)
    #          if artist['uri'] not in remix['artists'][0]['uri']]
    remixes = [remix for remix in get_remixes(artist)
              if artist['name'] in remix['name']]
    return remixes
    
def get_tracks_others_remixed(artist):
    # filter tracks by the artist others have remixed 
    def artist_uris(track):
        return [a['uri'] for a in track['artists']]
    
    remixes = [remix for remix in get_remixes(artist)
               if artist['uri'] in remix['artists'][0]['uri']]
    return remixes

In [ ]:
# Part 2: Print artist remixes 
# TODO: Add boards of canada test, should find 4
for artist in get_users_top_artists():
    remixes = get_all_remixes(artist)
    print artist['name'] ,': ', [t['name'] for t in remixes]

In [ ]:
# Test
# Find all remixes
boc_uri = 'spotify:artist:2VAvhf61GgLYmC6C8anyX1'
artist = sp.artist(boc_uri)
remixes = get_all_remixes(artist)
print artist['name']
print [t['name'] for t in remixes]

In [ ]:
# Test
# Find all artist tracks remixed by others
boc_uri = 'spotify:artist:4tZwfgrHOc3mvqYlEYSvVi'
artist = sp.artist(boc_uri)
remixes = get_tracks_others_remixed(artist)
print artist['name']
print [t['artists'][0]['name'] + '-' + t['name'] for t in remixes]

In [ ]:
# Test
# Find all tracks remixed by artist
boc_uri = 'spotify:artist:43mWhBXSflupNLuNjM5vff'
artist = sp.artist(boc_uri)
remixes = get_tracks_remixed_by(artist)
print artist['name']
print [t['artists'][0]['name'] + '-' + t['name'] for t in remixes]

In [ ]:
# Experiment
# Compare function performance with and without multiprocessingipcluster start -n 4

In [ ]:
$ ipcluster start -n 4

In [ ]:
t['name']

In [ ]:
class Test(dict):
    
    def __init__(self, a):
        dict.__init__(self, a)
    
    def cool_method(self):
        print 'hello world'
        
    def __str__(self):
        return ' '.join(self.keys())
a['awefaew'] = 2
b = Test(a)

In [ ]:
b.cool_method()

In [ ]:
a.keys()

In [ ]:
print b

In [ ]:
b.awefaew

In [ ]:
import os
import sys
sys.path.append('/Users/alexanderlitven/Projects/remix_finder/src')

In [ ]:
from spotify_adaptors import User, Artist, Track
reload(spotify_adaptors)
u = User()

In [ ]:
top_artists = u.get_top_artists()

In [ ]:
print([str(item) for item in top_artists])

In [ ]:
for artist in top_artists:
    print artist, "top tracks:"
    top_tracks = artist.get_top_tracks()
    print [str(t) for t in top_tracks]

In [ ]:
# Experiment
# Compare performance of parallel and non-parallel
# tasks for getting related artists
from joblib import Parallel, delayed

artists = get_all_users_top_artists(100)

def get_related(artists):
    return [get_related_artists(a) for a in artists] 

def get_related_parallel(artists):
    return Parallel(n_jobs=8)(delayed(get_related_artists) (a) for a in artists)

In [ ]:
#%time related_artists = get_related(artists)
%time related_artists = get_related_parallel(artists)

In [ ]:
artist_names = []
for a in related_artists:
    #print a['artists'][0]['name']
    for r in a['artists']:
        artist_names.append(r['name'])

In [ ]:
from collections import Counter

artist_count = Counter(artist_names)
artist_count.most_common(10)

In [ ]:
len(artist_names)

In [ ]:
artists = get_all_users_top_artists(100)
len(artists)

In [ ]:
# Test
# Transform spotify functions with limits
# Into unlimited
def fake_spotify_function(arg_0='start', limit=10, offset=0, arg_1='cool', arg_2='hehe'):
    for i in range(offset, offset+limit):
        print arg_0 + arg_1 + arg_2 + str(i)
        
def get_results(spotify_func, limit, num_results, **vargs):
    for i in range(0, num_results, limit):
        spotify_func(limit=limit, offset=i, **vargs)

In [ ]:
get_results(fake_spotify_function, 10, 20, arg_0='end', arg_1='hi', arg_2='lol')

In [ ]:
import spotify_adaptors
reload(spotify_adaptors)
from spotify_adaptors import Artist, User
artist = Artist.create_from_query('daft punk')

In [ ]:
user = User.create_from_scope('user-top-read')
artists = user.get_top_artists()[:20]  # TODO: Add similar artists

artist_remixes = []
artist_tracks_others_remixed = []

for artist in artists:
    print artist
    remixes = artist.get_remixes()
    other_artists_remixed = artist.get_tracks_others_remixed()
    artist_remixes.extend(remixes)
    artist_tracks_others_remixed.extend(other_artists_remixed)

remix_uris = set([r.uri for r in artist_remixes])
other_remixed_uris = set([r.uri for r in artist_tracks_others_remixed])

recommended_uris = remix_uris & other_remixed_uris

recommended_remixes = [r for r in artist_remixes if r.uri in recommended_uris]

In [ ]:
for r in recommended_remixes:
    print r

In [ ]:
pretty_lights = artists[4]

In [ ]:
pretty_lights_remixes = pretty_lights.get_remixes()

In [ ]:
print [r.name for r in remixes if 'Daft' in r.artist.name]

In [ ]:
daft_punk = artists[1]
daft_punk_others = daft_punk.get_tracks_others_remixed()

In [ ]:
print [r.name for r in others if 'Lights' in r.name]

In [ ]:
import spotify_adaptors
reload(spotify_adaptors)
from spotify_adaptors import Artist
artist = Artist.create_from_query('boards of canada')
remixes = artist.get_remixes()

In [ ]:
related_artists = artist.get_related_artists()

In [ ]:
len(related_artists)

In [ ]:
related_remixes = []
for rel_artist in related_artists:
    print rel_artist
    remixes = rel_artist.get_remixes()
    related_remixes.extend(remixes)

In [ ]:
clark = related_artists[8]
remixes = clark.get_remixes()

In [ ]:
len(remixes)

In [ ]:
related_remixes.sort(key=lambda r: r.popularity, reverse=True)
for t in related_remixes:
    print t.__str__(), t.popularity

In [ ]:
import spotify_adaptors
reload(spotify_adaptors)
from spotify_adaptors import User
user = User.create_from_scope('playlist-modify-public')

In [ ]:
from spotify_adaptors import Track

In [ ]:
t = Track.create_from_query('fake plastic trees')

In [ ]:
user.create_playlist([t], 'test_playlist')

In [ ]:
for i in range(0,10,10):
    print i